**Calculating p-values and directionality of protein intensity pattern using shifted geometric mean**
- Null hypothesis (H0): The true shifted geometric mean between two cell cycle stages are equal
- Alternative hypothesis (H1): The true shifted geometric mean between two cell cycle stages are different
- calculated Delta = difference in the shifted geometric mean, calculated from bootstrapped resamples of two cell cycle stages
- calculate p value, from 10000 Deltas, p-value=2*min(p,1-p), where p= (number of Delta > 0)/10000


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Date: Dec 2, 2025 (Bootstrap version)
Created on Thu Jun 13 10:04:12 2024
@author:  written by Darren Wethington, Indrani Nayak
Data: IL2 treated (24 h) + then NKG2D stimulated data on primary NK cell
This is done for various time points ['0','2','4','8','16','32','64','256']

1. This code performs BOOTSTRAP tests (n=10000) between protein data between (a) G1 and S, (b) S and G2 at a time point.
2. The result of 1 is summarized in the color map
3. This code plots protein abundances in terms of shifted geometric mean for all timepoints at G1, S and G2-M stages

Note: We did not get distinct M population here

Data processing steps:
- Negative values in raw data are replaced with 0 (untransformed data for bootstrap tests)

"""
import fcsextract
import numpy as np
from scipy.stats import ttest_ind, sem
import matplotlib.pyplot as plt
import pandas as pd



##read .facs fies as an matrix
def load_fcs_as_matrix(filename):
 """
  Loads flow cytometry data from an FCS file using fcsextract.

  Returns:
      tuple: A tuple containing two elements:
          - dict: A dictionary with metadata extracted from the FCS header.
          - list: A 2D list containing the event data (each sub-list represents an event).
  """
  # Use fcsextract to parse the FCS file
 metadata, data = fcsextract.fcsextract(filename)

 param_names = []
 for i in range(int(metadata[b'$PAR'])):
      param_names.append(metadata.get(b"$P%dS"%(i+1),metadata[b"$P%dN"%(i+1)]))
 data = np.asarray(data)
 
 
 #This part is added to prune the unwanted data ( The data had data1 and data 2 are same, for example barcode at column 53) Nov 5,24
 param_names=param_names[8:48]
 data=data[:,8:48]
 

 # #do the transformation of the raw data needed
 data[data<0]=0
 
#  print(f'minimum of data {np.min(data)}')
#  print(f'data shape {data.shape}')

 
 print(np.shape(data))

 return data,param_names

 
##analyse CD107 high population with gating for G1, S and G2
def analyze_data(path):
    #good_idx=[i for i in range(39)]
    time_strings = ['0','2','4','8','16','32','64','256']
    stage_strings = ['G1','S','G2']
    
    data,name_list = load_fcs_as_matrix(path+'2min_CD56dim_UMAP_G1.fcs')
    data,param_names=load_fcs_as_matrix(filename)

    #print(data.shape)#(20 x 54)
    #print(len(param_names))#54
    
    #54x6x2 size for a, b, c, and p_vals
    a = np.zeros((len(name_list),len(time_strings),len(stage_strings)-1),dtype='bool') #(54 x 8 x 2)
    b = np.zeros((len(name_list),len(time_strings),len(stage_strings)-1),dtype='bool')
    c = np.zeros((len(name_list),len(time_strings),len(stage_strings)-1),dtype='bool')
    p_vals = np.zeros((len(name_list),len(time_strings),len(stage_strings)-1))  # store p-values
    
    #print(a.shape)
    
    for time_idx,time in enumerate(time_strings): #[0,7]
        for protein_idx,name in enumerate(name_list): #[0,39]
            for i in range(len(stage_strings)-1):#2 [0,1] between [G1,S] and [S,G2]
                #print(time_idx, protein_idx,i)
                data1,names = load_fcs_as_matrix(path+time+'min_CD56dim_UMAP_'+stage_strings[i]+'.fcs')
                data2,names = load_fcs_as_matrix(path+time+'min_CD56dim_UMAP_'+stage_strings[i+1]+'.fcs')
                
             
                a[protein_idx,time_idx,i],b[protein_idx,time_idx,i],c[protein_idx,time_idx,i],p_vals[protein_idx,time_idx,i] = check_increase(data1[:,protein_idx:protein_idx+1],data2[:,protein_idx:protein_idx+1])

    return a,b,c,name_list,p_vals
# Remember, "a" is "has decrease", "b" is "has increase", and "c" is a raw comparison (no 



def check_increase(data1,data2): # this returns increase or decrease of specific protein between two cell stages
    """
    Bootstrap test for each protein between two stages (n=10000 iterations).
    
    - decrease[i] = True if mean1 > mean2 and bootstrap p-value < alpha
    - increase[i] = True if mean2 > mean1 and bootstrap p-value < alpha
    - comparison_result[i] = True if mean2 > mean1 (no significance check)
    - p_values[i] = the bootstrap p-value for this comparison
    """
    # Calculate shifted geometric mean for observed data
    delta = 1e-11
    mean1 = np.exp(np.mean(np.log(data1 + delta), axis=0)) - delta  # shifted geometric mean
    mean2 = np.exp(np.mean(np.log(data2 + delta), axis=0)) - delta  # shifted geometric mean
    
    # Use scalar booleans directly
    decrease = False
    increase = False
    
    alpha = 0.05
    n_bootstrap = 10000
    
    # Bootstrap resampling (data1 and data2 are single columns)
    bootstrap_diffs = np.zeros(n_bootstrap)
    
    for i in range(n_bootstrap):
        # Resample with replacement from each dataset
        resample1 = np.random.choice(data1[:,0], size=len(data1[:,0]), replace=True)
        resample2 = np.random.choice(data2[:,0], size=len(data2[:,0]), replace=True)
        
        
        shifted_geometric_mean_resample1 = np.exp(np.mean(np.log(resample1 + delta))) - delta
        shifted_geometric_mean_resample2 = np.exp(np.mean(np.log(resample2 + delta))) - delta
        
        # Calculate difference in means for this bootstrap sample for shifted GM mean
        bootstrap_diffs[i] = shifted_geometric_mean_resample2 - shifted_geometric_mean_resample1
    
    # Calculate two-tailed p-value
    # P-value is the proportion of bootstrap samples as extreme or more extreme than observed
    p_value = np.mean((bootstrap_diffs) >= 0)
    p_value = 2 * min(p_value, 1 - p_value)  # two-tailed
    
    # Determine decrease/increase based on p-value and mean comparison
    if p_value < alpha:
        if mean1 > mean2:
            decrease = True
        elif mean2 > mean1:
            increase = True
    
    comparison_result = bool(mean1 < mean2)  # mean2 greater than mean1 (no significance condition)
    
    # Return scalars
    return decrease, increase, comparison_result, p_value



# A plotting function that will plot average protein expression for a given protein for a 
# given dataset.  Provide the protein name as a string, and the dataset as a integer.
# I can provide dataset option context if needed; forgoing for now because datasets rely on
# my local path.
def plot_protein(protein_name,path):
    stage_strings = ['G1','S','G2']
    mean = np.zeros(len(stage_strings))
    std = np.zeros(len(stage_strings))

    time_strings = ['0','2','4','8','16','32','64','256']
    gray_shades = [
    '#F5F5F5',  # White Smoke
    '#DCDCDC',  # Gainsboro
    '#C0C0C0',  # Silver
    '#A9A9A9',  # Dark Gray
    '#808080',  # Gray
    '#696969',  # Dim Gray
    '#505050',  # Darker Gray
    '#303030']   # Very Dark Gray]
    #time_strings = ['2','8','16','32','64','256']
    mean = np.zeros((len(stage_strings),len(time_strings)+1)) #3 x 6
    std = np.zeros((len(stage_strings),len(time_strings)+1)) # 3 x 6
    data,name_list = load_fcs_as_matrix(path+'2min_CD56dim_UMAP_G1.fcs')
    idx = name_list.index(protein_name)
    for i in range(len(stage_strings)): #G1, S, G2
        data,name_list = load_fcs_as_matrix(path+'2min_CD56dim_UMAP_'+stage_strings[i]+'.fcs')
        
        #we do not need this step if the data is already log-transformed
        #data[data<0] = 0
   
        mean[i,0] = np.mean(data[:,idx])
        std[i,0] = sem(data[:,idx])
        
        #print(len(time_strings))
        for j in range(len(time_strings)): #time points
            data,names = load_fcs_as_matrix(path+time_strings[j]+'min_CD56dim_UMAP_'+stage_strings[i]+'.fcs')

            
            mean[i,j] = np.mean(data[:,idx])
            std[i,j] = sem(data[:,idx])
            
            
    #print(mean)
    #print(std)
    
    # Set global font settings
    plt.rcParams.update({
    'font.family': 'Arial',  # Use Arial font
    'font.size': 14,         # Set font size
    })
 
   
    plt.figure()
    plt.rcParams['figure.dpi'] = 600
    plt.rcParams['font.size'] = 18
    #plt.xlabel('Stage')
    #plt.ylabel('Average Protein Expression')
    # Access the Axes object
    ax = plt.gca()

    # Set the width of the x and y axes (spines)
    axes_width_size=2
    ax.spines['bottom'].set_linewidth(axes_width_size)
    ax.spines['left'].set_linewidth(axes_width_size)
    ax.spines['top'].set_linewidth(axes_width_size)  # Optional: set top spine width to 0 if not needed
    ax.spines['right'].set_linewidth(axes_width_size) 
    # Set the tick parameters to adjust tick width and font size
    plt.tick_params(axis='both', which='major', width=3, labelsize=20)
   
    #plt.ylim(1,50)
    plt.yscale('log') #Block it if doing in normal scale
    #plt.ylim(1,50)
    #plt.title(protein_name.decode('ascii'))
    ind = np.arange(3)
    # print(ind)
    # print(mean.shape)
    # print(mean)
  
    if len(mean.shape) == 1:
        plt.bar(['G1','S','G2-M'],mean,yerr=std)
    else:
        width = np.min(np.diff(ind))/(len(mean[0,:])+1)
        
        for i in range(len(mean[0,:])-1): # 6 time points
        
            #plotting geometrical average by taking exponential
            plt.bar(ind+width*i,np.exp(mean[:,i]),width=width,color=gray_shades[i % len(gray_shades)])
            plt.xticks(ind + ((len(mean[0,:])-1)/2)*width , ('G1', 'S', 'G2-M'))
            print(f'{i},{np.exp(mean[:,i])}')
    # Adjust legend size and position



    
    
# Which of the 9 possible conditions do our booleans generate??  Output used for figures.
# Remember, "a" is "has decrease", "b" is "has increase", and "c" is a raw comparison (no 
# statistical significance).  "c" is unused here.
def do_tests(a,b,c):
	#example: test 1 is that G1->S has a increase and S->G2 has a increase
    test1 = b[:,:,0] & b[:,:,1]
    # test 2 is that G1->S has a increase and S->G2 does not have a decrease or a increase
    test2 = b[:,:,0] & ~(a[:,:,1] | b[:,:,1])
    # etc
    test3 = b[:,:,0] & a[:,:,1] # increase -> decrease
    test4 = a[:,:,0] & b[:,:,1] # decrease -> increase
    test5 = a[:,:,0] & ~(a[:,:,1] | b[:,:,1])# dcrease -> flat
    test6 = a[:,:,0] & a[:,:,1] # decraese -> decrease
    test7 = ~(a[:,:,0] | b[:,:,0]) & b[:,:,1] #flat -> increase
    test8 = ~(a[:,:,0] | b[:,:,0]) & ~(a[:,:,1] | b[:,:,1]) #flat -> flat
    test9 = ~(a[:,:,0] | b[:,:,0]) & a[:,:,1] # #flat -> decrease
    return test1,test2,test3,test4,test5,test6,test7,test8,test9    # return a tuple


# This takes the output in the next function and converts it to a list of proteins in each
# category.  For example, if two decreases only happened for 4 proteins, it will return those
# protein names.  Not super useful anymore.
def give_names(test,names,protein_idx):
    name_str = []
    for name in names:
        name_str.append(name.decode('ascii'))
    pass_test = []
    for res in test:
        pass_test.append(np.asarray(name_str)[protein_idx][res[protein_idx].astype(bool)])
    return pass_test

# converts the results of all those booleans to a index for the figure result.  For instance,
# two succesive increases (test1) has index "0", an increase followed by "no change" (test2) 
# has index "1", etc.  "out" is the output, and it's given as an output argument.  The written
# file indicates which proteins belong to each category of trends.
def write_class_file(a,b,c,names):
    tests = do_tests(a,b,c)
    print(np.shape(tests)) #9
    out = np.zeros((len(tests[0][:,0]),len(tests[0][0,:])))
    #print(np.shape(out))
    for idx,test in enumerate(tests):
        out[test] = idx
    return out

    

# Performs two-tailed t-tests to determine if a significant change occurred between two stages.
# If the two-tailed p-value < 0.05, we check the direction of the means to classify as increase or decrease.
# Returns interpretation of the boolean results from the two-tailed tests.
def specific_protein_behavior(protein_name,timepoint, cell_cycle_stage):   
    # Find the index of the protein in the name_list
    protein_index = name_list.index(protein_name)
    #print(protein_index)
    # Extract the corresponding values from a, b, and c
    a_values = a[protein_index,timepoint,cell_cycle_stage]
    #print(a.shape)
    b_values = b[protein_index,timepoint,cell_cycle_stage]
    c_values = c[protein_index,timepoint,cell_cycle_stage]

    # Print the index and corresponding values for verification
    print(a_values,b_values,c_values)
    
    if (cell_cycle_stage==0):
        print("G1 to S:")
    else:
        print("S to G2:")
        
   # Check for significant increase or decrease (two-tailed test)
    if c_values.any():  # Check if any value in c_values is True
        if b_values.any():  # Check if any value in b_values is True
            print(f"There is a protein increase, two-tailed p < 0.05: {b_values}")
        else:
            print("There is a protein increase with no significance (two-tailed test) - no significant change between stages")
    else:
       if a_values.any():  # Check if any value in a_values is True
           print(f"There is a protein decrease, two-tailed p < 0.05: {a_values}")
       else:
           print("There is a protein decrease with no significance (two-tailed test) - no significant change between stages")





if __name__=='__main__':
    


    path='../data/primary/greg_umaps/new_untransformed/debarcode_IL2_NKG2D_'

    
    #Darrem's analyses with new gating with synthesis including only IDU positive but pH3 negative (no CD107+ pop)
    #path='./data/primary/Darren_analyses_redone_IN/debarcode_IL2_NKG2D_'
    

    filename= path+'2min_CD56dim_UMAP_G1.fcs' #note: These are CD56 dim cells
    #filename= './data/primary/CD107_cell_cycle_gates/debarcode_IL2_NKG2D_2min_G1.fcs' #note: These are CD56 dim cells
    data,param_name_list=load_fcs_as_matrix(filename)
    

    # a is a two dimesional array for all proteins, 1st column represents if proteins decrease between G1 and S and 2nd column represents the protein increase between S and G2 stages
    a,b,c,name_list,p_vals=analyze_data(path)

    #print(a)
    
    #writing the file for colormap
    test_idx = write_class_file(a,b,c,name_list)

    # print(test_idx)
    # print(test_idx.shape)
 
    # Ensure name_list is a list of strings
    name_list_str = [name.decode('ascii') if isinstance(name, bytes) else name for name in name_list]

    # Convert test_idx to a pandas DataFrame
    df_test_idx = pd.DataFrame(test_idx)

    # Add the name_list as the first column
    df_test_idx.insert(0, 'Protein Name', name_list_str)

    # Save the DataFrame to a CSV file (bootstrap test results)
    df_test_idx.to_csv('Primary_NK_NKG2D_colormap_untransformed_bootstrapping_Dec2_25.csv', index=False)
    print("Saved colormap CSV with bootstrap test results")
    
    # Create p-values DataFrame with detailed column names
    # p_vals shape: (n_proteins, n_timepoints, 2)
    # 2 transitions: [G1->S, S->G2]
    time_strings = ['0','2','4','8','16','32','64','256']
    p_val_columns = []
    p_val_data = []
    
    for time_idx, time in enumerate(time_strings):
        p_val_columns.append(f'{time}min_G1toS')
        p_val_columns.append(f'{time}min_StoG2')
    
    # Reshape p_vals from (n_proteins, n_timepoints, 2) to (n_proteins, n_timepoints*2)
    for protein_idx in range(len(name_list)):
        row = []
        for time_idx in range(len(time_strings)):
            row.append(p_vals[protein_idx, time_idx, 0])  # G1->S
            row.append(p_vals[protein_idx, time_idx, 1])  # S->G2
        p_val_data.append(row)
    
    df_pvals = pd.DataFrame(p_val_data, columns=p_val_columns)
    df_pvals.insert(0, 'Protein Name', name_list_str)
    
    # Save p-values to CSV
    df_pvals.to_csv('Primary_NK_NKG2D_pvalues_bootstrapping_Dec2_25.csv', index=False)
    print("Saved p-values CSV with bootstrap p-values for all proteins and timepoints")

    #these following part is added on Dec 1, 2025 to look at specific proteins of interest
    #---------------------------------------------------------------------------------------------
    # List of selected proteins to analyze
    selected_proteins = [
    b'CD7',
    b'CD8',
    b'CD57',
    b'CD96',
    b'NKG2D',
    b'CRACC',
    b'NKG2A',
    b'CD69',
    b'CD107a',
    b'CD161',
    b'CD11c',
    b'2B4',
    b'pCrkL',
    b'pPLCg2',
    b'pCreb',
    b'pSTAT5',
    b'pAkt',
    b'MAPKAPK2',
    b'EAT2',
    b'pSLP76',
    b'pNFkB',
    b'pErk1-2',
    b'pPlk1',
    b'p38',
    b'pLAT',
    b'pZAP70',
    b'pS6',
    b'Ki67',
    b'pVav1',
    b'DAP12',
    b'SAP'
]

    # Print the list of selected proteins
    print("\n=== Selected Proteins ===")
    for i, protein in enumerate(selected_proteins, 1):
        protein_str = protein.decode('ascii') if isinstance(protein, bytes) else protein
        if protein in name_list:
            idx = name_list.index(protein)
            print(f"{i}. {protein_str} (Index: {idx})")
        else:
            print(f"{i}. {protein_str} (NOT FOUND)")

    
    
    # Specify the protein name you are interested in
    # Filter dataframe to include only selected proteins and maintain the order
    selected_proteins_str = [p.decode('ascii') if isinstance(p, bytes) else p for p in selected_proteins]
    
    # Filter to get only selected proteins
    df_filtered = df_test_idx[df_test_idx['Protein Name'].isin(selected_proteins_str)].copy()
    
    # Create a mapping dictionary for the desired order
    protein_order = {protein: idx for idx, protein in enumerate(selected_proteins_str)}
    
    # Add a temporary column for sorting based on the desired order
    df_filtered['sort_order'] = df_filtered['Protein Name'].map(protein_order)
    
    # Sort by the order specified in selected_proteins
    df_filtered = df_filtered.sort_values('sort_order')
    
    # Remove the temporary sort column
    df_filtered = df_filtered.drop('sort_order', axis=1)
    
    df_filtered.to_csv('Primary_NK_NKG2D_selected_proteins_bootstrapping_Dec2_2025.csv', index=False)
    print(f"\nSaved {len(df_filtered)} proteins to 'Primary_NK_NKG2D_selected_proteins_bootstrapping_Dec2_2025.csv' (bootstrap test) in specified order")
    
    # Also filter and save p-values for selected proteins
    df_pvals_filtered = df_pvals[df_pvals['Protein Name'].isin(selected_proteins_str)].copy()
    df_pvals_filtered['sort_order'] = df_pvals_filtered['Protein Name'].map(protein_order)
    df_pvals_filtered = df_pvals_filtered.sort_values('sort_order')
    df_pvals_filtered = df_pvals_filtered.drop('sort_order', axis=1)
    
    df_pvals_filtered.to_csv('Primary_NK_NKG2D_selected_proteins_pvalues_bootstrapping_Dec2_2025.csv', index=False)
    print(f"Saved {len(df_pvals_filtered)} proteins' p-values to 'Primary_NK_NKG2D_selected_proteins_pvalues_bootstrapping_Dec2_2025.csv'")   




(5343, 40)
(5343, 40)
(5343, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(513, 40)
(466, 40)
(3120, 40)
(513, 40)
(51